In [2]:
from dotenv import load_dotenv, find_dotenv
print(find_dotenv())
load_dotenv()

C:\Worksapces\mini4\JoEunbi\.env


True

In [3]:
import os
os.getenv("LANGCHAIN_PROJECT")

'encore-ds36-turtorial'

In [16]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로드

loader = PyPDFLoader('./KQ85QNC850FXKR.pdf')
documents = loader.load()

In [17]:
import re

# 불용어 리스트 로드
with open('stopwords-ko.txt', 'r', encoding='utf-8') as f:
    korean_stopwords = set(f.read().splitlines())

In [18]:

# 특수문자, 불용어 제거
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    return ' '.join(word for word in text.split() if word not in korean_stopwords)  

# 개행문자 제거
for doc in documents:
    doc.page_content = preprocess_text(doc.page_content.replace('\n', ' '))
    doc.metadata = {}

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, # 기본값 1000
    chunk_overlap=20
)
docs = splitter.split_documents(documents)

In [20]:
for doc in documents:
    print(doc.page_content)

KOR 사용 설명서 본 제품은 대한민국에서만 사용하도록 만들어져 있으며 해외에서는 품질을 보증하지 않습니다 FOR KOREA UNIT STANDARD ONLY 안전을 위한 주의 사항은 사용자의 안전을 지키고 재산상의 손해 등을 막기 위한 내용입니다 읽고 올바르게 사용해 주세요 사용 설명서 상의 그림은 설명을 위해 그려진 것으로 실제의 모양과 부분적으로 다를 수 있습니다 제품 디자인 제품 규격은 제품 성능 향상을 위해 사전 예고 없이 변경될 수 있습니다 2023 삼성전자 주식회사 본 사용 설명서의 저작권은 삼성전자주에 있습니다 삼성전자주의 사전 허가 없이 사용 설명서 내용의 일부 또는 전부를 무단 사용하거나 복제하는 것은 금지되어 있습니다 사용 설명서를 읽기 전에 본 제품의 설명서는 사용 설명서와 e설명서로 구성되어 있습니다 메뉴 설정 고객지원 e설명서 보기 고객지원 센터에서 문서 형태의 설명서를 다운로드 받아 PC 또는 모바일에서 이용해 보세요 httpswwwsamsungcomsecsupport 외부기기 연결 방법은 메뉴 연결된 기기 연결 가이드를 참조하세요 안전을 위한 주의 사항 사용자의 안전을 지키고 재산상의 손해를 막기 위한 내용입니다 잘 읽고 제품을 올바르게 사용해 주세요 안전을 위한 주의 사항에 사용되는 기호 기호 명칭 의미 경고 지키지 않았을 경우 사용자가 사망하거나 중상을 입을 수 있습니다 주의 지키지 않았을 경우 사용자가 부상을 당하거나 재산상의 손해를 입을 수 있습니다 금지 행위 금지 표시로 하지 말아야 할 항목을 강조하여 나타냅니다 지시 강제 지시 표시로 꼭 따라야 할 항목을 강조하여 나타냅니다 제품에 표기된 기호는 아래의 표에서 의미를 확인할 수 있습니다 기호 의미 제품 내부에 고전압이 흐르므로 내부 고전압 부품과의 접촉으로 위험이 발생할 수 있음을 표시합니다 기호는 제품 사용 유지 보수에 대한 자료가 포함되어 있음을 나타냅니다 Class II 제품 기호는 안전 접지가 필요 없는 제품임을 나타냅니다 벽면 콘센트에 연결하여 사용하는 제품에 기

In [21]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model='text-embedding-3-small', 
)

In [ ]:
import pandas as pd
from langchain.schema import Document

repair_costs_df = pd.read_csv('TV_repair.csv')

In [35]:
repair_costs_df['평균가'] = repair_costs_df['평균가'].str.replace(',', '')

In [36]:

repair_docs = [
    Document(
        page_content=preprocess_text(
            f"{row['모델명']} ({row['이름']})의 {row['부품']} 수리비용은 약 {row['평균가']}원 입니다."
        )
    )
    for _, row in repair_costs_df.iterrows()
]

In [37]:
from langchain_chroma.vectorstores import Chroma

all_docs = docs + repair_docs
vector_store = Chroma.from_documents(all_docs, embeddings)

In [38]:
query = '제품 사용 방법 알려주세요.'

retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={'k':3} 
)
retrievals = retriever.batch([query])
retrievals

[[Document(metadata={}, page_content='내용입니다 잘 읽고 제품을 올바르게 사용해 주세요 안전을 위한 주의 사항에 사용되는 기호 기호 명칭 의미 경고 지키지 않았을 경우 사용자가 사망하거나 중상을 입을 수 있습니다'),
  Document(metadata={}, page_content='내용입니다 잘 읽고 제품을 올바르게 사용해 주세요 안전을 위한 주의 사항에 사용되는 기호 기호 명칭 의미 경고 지키지 않았을 경우 사용자가 사망하거나 중상을 입을 수 있습니다'),
  Document(metadata={}, page_content='전부를 무단 사용하거나 복제하는 것은 금지되어 있습니다 사용 설명서를 읽기 전에 본 제품의 설명서는 사용 설명서와 e설명서로 구성되어 있습니다 메뉴 설정 고객지원 e설명서 보기')]]

In [40]:
# 탬플릿 생성
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ('system', '''
    당신은 제품 사용설명서의 정보를 바탕으로 사용자 문의에 답변하는 전문적인 AI 챗봇입니다. 
    답변은 항상 친절하고 사용자가 이해하기 쉽도록 간결해야 합니다.
    필요한 경우 추가 정보를 요청하도록 안내합니다.
    '''),
    ('user', '''
    사용자의 질문에 아래 제공된 context만 사용하여 답변해주세요. 
    context 외의 추측이나 추가 정보를 제공하지 마세요.
    
    **질문 예시**:
    - 제품의 특정 기능을 어떻게 사용하나요?
    - 제품의 디스플레이 수리 비용은 얼마인가요?
    
    question: {query}
    context: {context}
    ''')
])

prompt

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\n    당신은 제품 사용설명서의 정보를 바탕으로 사용자 문의에 답변하는 전문적인 AI 챗봇입니다. \n    답변은 항상 친절하고 사용자가 이해하기 쉽도록 간결해야 합니다.\n    필요한 경우 추가 정보를 요청하도록 안내합니다.\n    '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template='\n    사용자의 질문에 아래 제공된 context만 사용하여 답변해주세요. \n    context 외의 추측이나 추가 정보를 제공하지 마세요.\n    \n    **질문 예시**:\n    - 제품의 특정 기능을 어떻게 사용하나요?\n    - 제품의 디스플레이 수리 비용은 얼마인가요?\n    \n    question: {query}\n    context: {context}\n    '), additional_kwargs={})])

In [41]:
prompt.invoke({'query': query, 'context': retriever})

Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)
Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)


ChatPromptValue(messages=[SystemMessage(content='\n    당신은 제품 사용설명서의 정보를 바탕으로 사용자 문의에 답변하는 전문적인 AI 챗봇입니다. \n    답변은 항상 친절하고 사용자가 이해하기 쉽도록 간결해야 합니다.\n    필요한 경우 추가 정보를 요청하도록 안내합니다.\n    ', additional_kwargs={}, response_metadata={}), HumanMessage(content="\n    사용자의 질문에 아래 제공된 context만 사용하여 답변해주세요. \n    context 외의 추측이나 추가 정보를 제공하지 마세요.\n    \n    **질문 예시**:\n    - 제품의 특정 기능을 어떻게 사용하나요?\n    - 제품의 디스플레이 수리 비용은 얼마인가요?\n    \n    question: 제품 사용 방법 알려주세요.\n    context: tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000214329E1EB0> search_kwargs={'k': 3}\n    ", additional_kwargs={}, response_metadata={})])

In [42]:

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o',
    temperature=0.5
)

In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = {'query': RunnablePassthrough(), 'context': retriever} | prompt | model | StrOutputParser()

chain.invoke(query)

'\n제품 사용 방법에 대한 구체적인 정보는 제공된 context에 포함되어 있지 않습니다. 제품의 사용 설명서를 자세히 읽어보시거나 e설명서를 확인하시면 도움이 될 것입니다. 추가적인 정보가 필요하시면 고객 지원을 통해 문의해 주세요.'

In [44]:
chain.invoke("인공지능 기능은 어떻게 사용해요?")

"인공지능 기능을 사용하려면 먼저 리모컨의 왼쪽 방향 버튼을 눌러 설정 메뉴로 이동하세요. 그 다음, 전체 설정에서 일반 탭으로 이동하여 '개인정보' 항목을 선택한 후, '인공지능 AI 모드'를 실행하시면 됩니다. 이 모드는 주변 환경과 사용 패턴을 분석하여 최적의 시청 경험을 제공합니다. 추가 도움이 필요하시면 말씀해 주세요!"

In [45]:
chain.invoke("디스플레이 수리비용은 얼마인가요?")

'\n죄송하지만 제공된 정보로는 디스플레이 수리 비용에 대한 내용을 찾을 수 없습니다. 제품의 디스플레이 수리 비용에 대한 자세한 정보를 원하신다면, 고객 서비스 센터에 문의하시거나 제품 보증서 혹은 공식 웹사이트를 참조하시기 바랍니다. 추가적인 도움이 필요하시면 언제든지 말씀해 주세요.'